In [ ]:
from pathlib import Path
import shutil

scratchpad_dir = Path("../scratchpad")

# Subdirectories to clean
subdirs = ["plots", "data", "notes"]

cleaned = []
for subdir in subdirs:
    folder = scratchpad_dir / subdir
    if folder.exists():
        # Remove all files in the folder
        for item in folder.iterdir():
            if item.is_file():
                item.unlink()
                cleaned.append(str(item))
            elif item.is_dir():
                shutil.rmtree(item)
                cleaned.append(str(item) + "/")
    else:
        # Create the folder if it doesn't exist
        folder.mkdir(parents=True, exist_ok=True)

if cleaned:
    print(f"🧹 Cleaned {len(cleaned)} items from scratchpad:")
    for item in cleaned:
        print(f"   - {item}")
else:
    print("✨ Scratchpad folders are already empty")

print(f"\n📁 Empty folders preserved:")
for subdir in subdirs:
    print(f"   - scratchpad/{subdir}/")

# Web Research Sub-Agent Test Notebook

This notebook tests the web research sub-agent, which is responsible for:
- **Finding Information**: Searching for relevant, reliable sources
- **Gathering Data**: Collecting facts, statistics, and quotes
- **Documenting Sources**: Keeping detailed records of where information came from
- **Assessing Initial Quality**: Noting if sources seem reliable or questionable

The main agent uses this sub-agent to:
- Research why stock prices are moving
- Analyze recent price action, volume, and volatility
- Investigate company-specific news, earnings, and guidance
- Gather sector trends, competitor info, and macro factors
- Collect datasets for the analysis agent to process

## Setup

In [ ]:
import sys
import importlib

sys.path.insert(0, '..')  # Add parent directory to path

# Import and reload to pick up any changes to the agent
from agents import web_research_agent
importlib.reload(web_research_agent)
from agents.web_research_agent import web_research_agent_graph

from langgraph.types import Command
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ Web research agent loaded successfully (with latest changes)")

## Helper Function to Test the Agent

In [ ]:
def test_web_research_agent(user_message: str, thread_id: str = "test-thread"):
    """
    Test the web research agent with a research request.
    
    Args:
        user_message: The research task/question for the agent
        thread_id: Unique thread ID for this conversation
    
    Returns:
        The agent's final response message
    """
    print("="*80)
    print(f"RESEARCH REQUEST:\n{user_message}")
    print("="*80)
    print("\nResearching...\n")
    
    # Invoke the agent with a fresh conversation (NOT resuming)
    result = web_research_agent_graph.invoke(
        {"messages": [("user", user_message)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    # Get the final AI message
    final_message = result["messages"][-1].content
    
    print("="*80)
    print("WEB RESEARCH AGENT FINDINGS:")
    print("="*80)
    print(final_message)
    print("\n" + "="*80)
    print("✅ Research completed. Check deep-agent/scratchpad/data/ for any saved datasets.")
    print("✅ Check deep-agent/scratchpad/notes/ for any detailed notes.")
    print("="*80 + "\n")
    
    return final_message

---
# Example 1: Single Stock Research (Simple)

**Context**: A trader wants to understand why a specific stock moved significantly.

**Sub-agent role**: Research recent news and events related to the stock and provide cited findings.

In [ ]:
# Example 1: Single stock research
example_1_message = """Research why Microsoft (MSFT) stock has been moving recently.

Focus on:
- Recent news or announcements
- Any earnings or guidance updates
- AI-related developments

Provide your findings with proper source citations.
"""

# Use a fresh thread ID to avoid any cached responses
import time
response_1 = test_web_research_agent(example_1_message, thread_id=f"example-1-{int(time.time())}")

---
# Example 2: Sector Analysis (Medium)

**Context**: A portfolio manager wants to understand sector-wide trends affecting their holdings.

**Sub-agent role**: Research the semiconductor sector, identify key trends, and gather relevant data.

In [ ]:
# Example 2: Sector analysis
example_2_message = """Research the current state of the semiconductor sector.

I need to understand:
1. How major semiconductor stocks have performed recently (NVDA, AMD, INTC, AVGO)
2. Key trends driving the sector (AI demand, supply chain, geopolitics)
3. Recent analyst sentiment and price target changes
4. Any upcoming catalysts or risks

Save any relevant data you find to the scratchpad/data/ folder.
Provide properly cited sources for all claims.
"""

response_2 = test_web_research_agent(example_2_message, thread_id=f"example-2-{int(time.time())}")

---
# Example 3: Comprehensive Event Research (Complex)

**Context**: A risk manager needs to understand the market implications of a major economic event.

**Sub-agent role**: Research across multiple asset classes and provide a comprehensive view with data.

In [ ]:
# Example 3: Comprehensive event research
example_3_message = """Research the market impact of the most recent Federal Reserve interest rate decision.

I need comprehensive research covering:

1. **The Decision Itself**
   - What was the decision (rate change, guidance)
   - How did it compare to market expectations
   - Key quotes from Fed Chair's statement

2. **Market Reactions Across Asset Classes**
   - Equity indices (S&P 500, Nasdaq, small caps)
   - Bond yields (2Y, 10Y treasuries)
   - Dollar index and gold

3. **Sector-Specific Impacts**
   - Which sectors benefited vs suffered
   - Rate-sensitive stocks (banks, REITs, utilities)
   - Growth vs value performance

4. **Forward-Looking Implications**
   - Market pricing for future rate decisions
   - Analyst commentary on the path forward
   - Risks and uncertainties

Save any relevant data or notes to the scratchpad folders.
Every claim must have a source citation.
"""

response_3 = test_web_research_agent(example_3_message, thread_id=f"example-3-{int(time.time())}")

---
# Verify Output Files

Check what data and notes were saved to the scratchpad directories.

In [ ]:
import os
from pathlib import Path

scratchpad_dir = Path("../scratchpad")

# Check data directory
data_dir = scratchpad_dir / "data"
if data_dir.exists():
    data_files = list(data_dir.glob("*"))
    if data_files:
        print(f"📊 Found {len(data_files)} files in scratchpad/data/:")
        for f in sorted(data_files, key=lambda x: x.stat().st_mtime, reverse=True):
            print(f"  - {f.name}")
    else:
        print("📊 scratchpad/data/ is empty")

print()

# Check notes directory
notes_dir = scratchpad_dir / "notes"
if notes_dir.exists():
    notes_files = list(notes_dir.glob("*"))
    if notes_files:
        print(f"📝 Found {len(notes_files)} files in scratchpad/notes/:")
        for f in sorted(notes_files, key=lambda x: x.stat().st_mtime, reverse=True):
            print(f"  - {f.name}")
    else:
        print("📝 scratchpad/notes/ is empty")

---
# Notes

## Expected Outputs

For each example, the web research agent should provide:

### 1. Summary of Findings
- Each statement with inline citation: [Source Name](URL)

### 2. Key Facts with Citations
- Format: "Fact statement [Source Name](URL)"
- No uncited claims

### 3. Citations List
- Numbered list of all sources used
- Brief description of what info came from each source

### 4. Gaps in Research
- What couldn't be found or verified

## Data Output

When the agent finds useful data, it should save to:
- `scratchpad/data/` - CSV files, datasets for analysis
- `scratchpad/notes/` - Detailed research notes

## Best Practices Being Tested

- Multiple search queries to triangulate information
- Noting the date of sources (recency matters)
- Distinguishing between news, opinion, and research
- Flagging unreliable sources

## Integration with Main Agent

In production, the main agent would:
1. Assign research tasks to `web-research-agent`
2. Receive findings with citations
3. Send findings to `credibility-agent` for verification
4. Pass verified data to `analysis-agent` for processing
5. Compile everything into final reports